In [257]:
# Hermawan - NEVTIK
import requests
import pandas
import csv

In [258]:
shopee_url = "https://shopee.co.id/"
keyword_search = "Iphone"
limit = 100  # Max 100 (bug)
official = 1 # Change value to 1 for filterring official store

if official == 1:
    official_shop = "ShopeeMall"
else:
    official_shop = "Shopee"

csv_file = open('{}_{}_{}.csv'.format(official_shop,keyword_search,limit),'w')
csv_writer = csv.writer(csv_file)

In [259]:
headers = {
    #Change 'User-Agent' equivalent to your browser
    'User-Agent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:73.0) Gecko/20100101 Firefox/73.0",
    'referrer' : '{}search?keyword={}'.format(shopee_url,keyword_search)
}

In [260]:
product_list,sold_list,price_list,brand_list,rating,stock,location,price_before_discount = [],[],[],[],[],[],[],[]
columns = ["Product","Brand","Location","Sold","Stock","Rating","Price_Before_Discount","Price"]

class GetID:
    def __init__(self,search,limit,official):
        API_id_url = "https://shopee.co.id/api/v2/search_items/?by=relevancy&keyword={}&limit={}&newest=0&official_mall={}&order=desc&page_type=search&version=2".format(search,limit,official)
        self.api_requests = requests.get(API_id_url,headers = headers).json()
        self.shop_id,self.item_id = [],[]
        for item in self.api_requests["items"]:
            self.item_id.append(item["itemid"])
            self.shop_id.append(item["shopid"])
            
get_item_id = GetID(keyword_search,limit,official).item_id
get_shop_id = GetID(keyword_search,limit,official).shop_id

def get_shopee():
    for index in range(len(get_item_id)):
        API_item_url = "https://shopee.co.id/api/v2/item/get?itemid={}&shopid={}".format(get_item_id[index],get_shop_id[index])
        api_requests = requests.get(API_item_url, headers=headers).json()
        yield api_requests

for i in get_shopee():
    item = i['item']
    product_list.append(item['name'])
    sold_list.append(item['historical_sold'])
    price_list.append(int(str(item['price_min'])[:-5]))
    location.append(item['shop_location'])
    rating.append(round(float(item['item_rating']['rating_star']),1))
    stock.append(item['stock'])
    brand_list.append(item['brand'])
    desc_list.append(item['description'])
    try: 
        price_before_discount.append(int(str(item['price_before_discount'])[:-5]))
    except: 
        price_before_discount.append(int(str(item['price_min'])[:-5]))

In [261]:
product_detail = {"Product":product_list,"Brand":brand_list,"Location":location,"Sold":sold_list,"Stock":stock,"Rating":rating,"Price_Before_Discount":price_before_discount,"Price":price_list}

csv_writer.writerow(columns)

for item_to_csv in range(len(product_list)):
    csv_writer.writerow([product_detail["Product"][item_to_csv],
                         product_detail["Brand"][item_to_csv],
                         product_detail["Location"][item_to_csv],
                         product_detail["Sold"][item_to_csv],
                         product_detail["Stock"][item_to_csv],
                         product_detail["Rating"][item_to_csv],
                         product_detail["Price_Before_Discount"][item_to_csv],
                         product_detail["Price"][item_to_csv]
                        ])
    
csv_file.close()
dataframe = pandas.DataFrame(product_detail,columns=columns)
dataframe

,Product,Brand,Location,Sold,Stock,Rating,Price_Before_Discount,Price
0,Kabel data JOYSEUS 3A Lightning Apple Iphone C...,Joyseus,KOTA JAKARTA UTARA,30,485,4.8,99000,39900
1,Kabel data / Cable data Apple Lighting Cable I...,Joyseus,KOTA JAKARTA UTARA,716,122,4.8,49900,35000
2,Apple iPhone Xr 256GB - Garansi Resmi 1 Tahun,None,KOTA JAKARTA UTARA,2,2,5.0,13999000,13999000
3,Moshi iPhone 6/6s iGlaze,,KOTA JAKARTA SELATAN,172,103,4.7,399000,159600
4,Moshi iPhone 6/6s iGlaze Armour,0,KOTA JAKARTA SELATAN,12,38,5.0,529000,529000
...,...,...,...,...,...,...,...,...
95,Log On Kabel Iphone 5/6/7 2.4A L22 3M,Log-On,KOTA JAKARTA PUSAT,309,2206,4.8,37500,37500
96,Peonia Soft Case iPhone 6 6s 6 Plus 6+ 6s Plus...,Peonia,KOTA BEKASI,181,275,4.9,50000,12900
97,Tempered Glass TG All Tipe Type Handphone Hp C...,None,KOTA JAKARTA BARAT,208,7786,4.5,20000,15000
98,ESR iPhone 11 Pro Max Tempered Glass Case with...,ESR,Luar Negeri,245,152,5.0,248520,248520
